# Name-Entity Recognition of Descriptoins of Various Products

## Part III: Testing the Model

## 1. Dataset

In [1]:
import pandas as pd
import numpy as np
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm 
from spacy.util import minibatch, compounding
from spacy import displacy
import string
import pickle
from spacy.gold import GoldParse
from spacy.scorer import Scorer

In [2]:
df = pd.read_csv("data.csv")

In [3]:
df.shape

(671621, 8)

In [4]:
df.head()

,title,description,summary,brand,price,meta,provider_category,provider
0,"adidas Originals - Superstar - Valkoinen - US 5,5",NaN,NaN,adidas Originals,66.50,"{""SIZE"": [""us 5,5""], ""COLOR"": [""valkoinen""], ""...",17-muoti-ja-vaatetus,Caliroots
1,Sc-Erna Polvipituinen Hame Sininen Soyaconcept,"SOYACONCEPT on tanskalainen brändi, joka luo e...",NaN,Soyaconcept,49.99,"{""SIZE"": [""36""], ""COLOR"": [""cristal blue""], ""G...",17-muoti-ja-vaatetus,Boozt
2,Dana Buchman Silmälasit Taren CARAMEL TORTOISE,Dana Buchman Taren Silmälasit. Collection:Men....,NaN,Dana Buchman,146.00,"{""SIZE"": [""54""], ""COLOR"": [""tortoise""], ""GENDE...",13-silmalasit-ja-piilolinssit,Smartbuy Glasses
3,Active Sports Woven Shorts B Shortsit Musta PUMA,PUMA Active Sports Woven Shorts B,NaN,PUMA,27.00,"{""SIZE"": [""164"", ""128"", ""110"", ""116"", ""104"", ""...",17-muoti-ja-vaatetus,Boozt
4,Renata Polvipituinen Hame Musta Fall Winter Sp...,Fall Winter Spring Summer. A-linjainen.,NaN,Fall Winter Spring Summer,199.00,"{""SIZE"": [""xs""], ""COLOR"": [""jet black""], ""GEND...",17-muoti-ja-vaatetus,Boozt


In [5]:
i=0
print(df.loc[i])
print(df['meta'].loc[i])

title                adidas Originals - Superstar - Valkoinen - US 5,5
description                                                        NaN
summary                                                            NaN
brand                                                 adidas Originals
price                                                             66.5
meta                 {"SIZE": ["us 5,5"], "COLOR": ["valkoinen"], "...
provider_category                                 17-muoti-ja-vaatetus
provider                                                     Caliroots
Name: 0, dtype: object
{"SIZE": ["us 5,5"], "COLOR": ["valkoinen"], "GENDER": ["unisex"]}


In [6]:
i=1
print(df.loc[i])
print(df['meta'].loc[i])

title                   Sc-Erna Polvipituinen Hame Sininen Soyaconcept
description          SOYACONCEPT on tanskalainen brändi, joka luo e...
summary                                                            NaN
brand                                                      Soyaconcept
price                                                            49.99
meta                 {"SIZE": ["36"], "COLOR": ["cristal blue"], "G...
provider_category                                 17-muoti-ja-vaatetus
provider                                                         Boozt
Name: 1, dtype: object
{"SIZE": ["36"], "COLOR": ["cristal blue"], "GENDER": ["women"]}


## 2. Entity and Entity Types

One can obtain entity and entity types from the *meta* and *brand* columns. However, there are many words incorrectly written entities. These are deleted, and many entities obtained from the *title* and *descriptoin* columns are added by hand. The saved entities and eneity types are uploaded below.

In [7]:
with open("entity.txt", "rb") as fp:   # Unpickling
    entity = pickle.load(fp)

In [8]:
with open("entity_types.txt", "rb") as fp:   # Unpickling
    entity_types = pickle.load(fp)

In [9]:
with open("titles_unique_brand.txt", "rb") as fp:   # Unpickling
    titles_unique_brand = pickle.load(fp)

In [10]:
with open("description_unique_brand.txt", "rb") as fp:   # Unpickling
    description_unique_brand = pickle.load(fp)

In [11]:
entity[0:10]

['US 5,5',
 'Valkoinen',
 '36',
 'Cristal Blue',
 '54',
 'Tortoise',
 '164',
 '128',
 '110',
 '116']

In [12]:
entity_types[0:10]

['SIZE',
 'COLOR',
 'SIZE',
 'COLOR',
 'SIZE',
 'COLOR',
 'SIZE',
 'SIZE',
 'SIZE',
 'SIZE']

## 3. Training Data and Model

- Loop through the title column, and the description column, make a list extracted from title/description column which have unique brands.
    - For the description column, we can drop rows include `&` and `<b>` signs, in order to increase the accuracy.
- In each loop, use the entity and entity types lists:
    - Construct annotation, but this annotation will be highly overlapped in the charactors' ranges. This is not allowed in the training process. 
    - Get rid of overlapped charactors' ranges, always choose the phrases with wider range
    - Repeating the same process above, in case there are still overlapped charactors' ranges. This is due to the fact that, the wirtten code, only compares two phrases at once, and this may lead to some left over overlapping.
- The saved data is uploaded below.

In [13]:
with open("TRAIN_DATA.txt", "rb") as fp:   # Unpickling
    TRAIN_DATA = pickle.load(fp)

In [14]:
# upload the model to output directory
output_dir=Path("/Users/farukakbar/Desktop/ner_products/model")
if output_dir is not None:
    print("Loading from", output_dir)
    nlp = spacy.load(output_dir)
    #for text, _ in TRAIN_DATA:
    #    doc = nlp(text)
    #    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    #    #print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Loading from /Users/farukakbar/Desktop/ner_products/model


## 4. Results

In [15]:
# Entity and Entity Types output
def ent_output(nlp_doc):
    ent_dict={}
    entity_list=list(doc.ents) 
    for item in entity_list:
        ent_dict[item]=item.label_
        #print(item, item.label_)
    print(ent_dict)

### 1) Results on the trained data

In [16]:
trained_data=[]
for n in range(0,100):
    doc = nlp(TRAIN_DATA[n][0])
    spacy.displacy.render(doc, style="ent")
    ent_output(doc)
    trained_data.append(TRAIN_DATA[n])

{Custom: 'BRAND'}


{Svartzonker: 'BRAND'}


{40: 'SIZE', black: 'COLOR'}


{Mercury: 'BRAND', harmaa/musta: 'COLOR'}


{Grunt: 'BRAND'}


...

{black: 'COLOR', naisten: 'GENDER', 40: 'SIZE'}


{Bulldog: 'BRAND', Bamboo: 'COLOR'}


{Vicomte A.: 'BRAND'}


{Stanton Street Sports: 'BRAND', Sininen: 'COLOR'}


{Silhouette: 'BRAND', Titan: 'BRAND'}


### 2) Result on the unseen data

In [17]:
for n in range(0,100):
    if df['title'].loc[n] not in titles_unique_brand:
        doc = nlp(df['title'].loc[n])
        spacy.displacy.render(doc, style="ent")
        ent_output(doc)

{White: 'COLOR'}


/Applications/anaconda2/envs/Python3/lib/python3.8/site-packages/spacy/displacy/__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


{}


{Black/Br: 'COLOR'}


{Diesel: 'BRAND'}


...

{40ml: 'SIZE'}


{valkoinen: 'COLOR'}


{Red: 'COLOR'}


{Dana Buchman: 'BRAND', BLACK: 'COLOR'}


{Roberto Cavalli: 'BRAND'}


In [18]:
for n in range(0,200):
    if type(df['description'].loc[n])==str:
        if df['description'].loc[n] not in description_unique_brand:
            doc = nlp(df['description'].loc[n])
            spacy.displacy.render(doc, style="ent")
            ent_output(doc)

{}


{New Balance: 'BRAND'}


{Diesel: 'BRAND', Shiny Black.: 'COLOR', Plastic.: 'MATERIAL', 54.: 'SIZE'}


{}


{}


...

{Tommy Hilfiger luo: 'BRAND'}


{naisten: 'GENDER'}


{Police: 'BRAND', Kids: 'AGE_GROUP', Blue.: 'COLOR', Plastic.: 'MATERIAL', 48.: 'SIZE'}


{}


{}


### 3) Arbirary texts

In [19]:
doc = nlp('I have adidas Originals White sneakers.')
spacy.displacy.render(doc, style="ent")
ent_output(doc)

{adidas: 'BRAND', White: 'COLOR'}


In [20]:
doc = nlp('You have Soyaconcept Blue coat.')
spacy.displacy.render(doc, style="ent")
ent_output(doc)

{Soyaconcept: 'BRAND', Blue: 'COLOR'}


In [21]:
doc = nlp('I have a Black shirt, which is 3 kg.')
spacy.displacy.render(doc, style="ent")
ent_output(doc)

{Black: 'COLOR', 3 kg.: 'WEIGHT'}


## 5. Accuracy

In [22]:
def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot['entities'])
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

The explanations of the scores:
- ents_p: Named entity accuracy (precision).
- ents_r: Named entity accuracy (recall).
- ents_f: Named entity accuracy (F-score).
- ents_per_type: Scores per entity label. Keyed by label, mapped to a dict of p, r and f scores.
- token_acc: Tokenization accuracy.

### 1) Scores on the Trained Data

In [23]:
results = evaluate(nlp, trained_data)

In [24]:
results

{'uas': 0.0,
 'las': 0.0,
 'las_per_type': {'': {'p': 0.0, 'r': 0.0, 'f': 0.0}},
 'ents_p': 100.0,
 'ents_r': 100.0,
 'ents_f': 100.0,
 'ents_per_type': {'BRAND': {'p': 100.0, 'r': 100.0, 'f': 100.0},
  'SIZE': {'p': 100.0, 'r': 100.0, 'f': 100.0},
  'COLOR': {'p': 100.0, 'r': 100.0, 'f': 100.0},
  'WEIGHT': {'p': 100.0, 'r': 100.0, 'f': 100.0},
  'GENDER': {'p': 100.0, 'r': 100.0, 'f': 100.0},
  'MATERIAL': {'p': 100.0, 'r': 100.0, 'f': 100.0}},
 'tags_acc': 0.0,
 'token_acc': 100.0,
 'textcat_score': 0.0,
 'textcats_per_cat': {}}

### 2) Scores on the Unseen Data

In [25]:
with open("test_data.txt", "rb") as fp:   # Unpickling
    test_data = pickle.load(fp)

In [26]:
len(test_data)

1414

In [27]:
results = evaluate(nlp, test_data)

In [28]:
results

{'uas': 0.0,
 'las': 0.0,
 'las_per_type': {'': {'p': 0.0, 'r': 0.0, 'f': 0.0}},
 'ents_p': 82.85446595877576,
 'ents_r': 88.16882685277501,
 'ents_f': 85.42907744324584,
 'ents_per_type': {'SIZE': {'p': 80.65296251511487,
   'r': 85.18518518518519,
   'f': 82.85714285714286},
  'GENDER': {'p': 97.85714285714285,
   'r': 98.56115107913669,
   'f': 98.2078853046595},
  'BRAND': {'p': 80.19884009942005,
   'r': 86.19768477292965,
   'f': 83.09012875536482},
  'COLOR': {'p': 86.49789029535864,
   'r': 90.04392386530014,
   'f': 88.23529411764706},
  'AGE_GROUP': {'p': 100.0, 'r': 94.11764705882352, 'f': 96.96969696969697},
  'MATERIAL': {'p': 85.17241379310346,
   'r': 94.6360153256705,
   'f': 89.65517241379311},
  'WEIGHT': {'p': 27.27272727272727, 'r': 100.0, 'f': 42.857142857142854}},
 'tags_acc': 0.0,
 'token_acc': 100.0,
 'textcat_score': 0.0,
 'textcats_per_cat': {}}